In [16]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import yfinance as yf
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

def corrigir_dataframe(df):
    """Corrige problemas com MultiIndex e garante formato consistente"""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    return df

def processar_dados(ticker):
    """Obtém e processa dados do Yahoo Finance"""
    try:
        dados = yf.download(ticker + '.SA', period='1y', progress=False)
        dados = corrigir_dataframe(dados)
        
        if dados.empty:
            raise ValueError("Nenhum dado obtido")
            
        # Renomear colunas e garantir formato correto
        dados = dados.rename(columns={
            'Open': 'open',
            'High': 'high', 
            'Low': 'low',
            'Close': 'close',
            'Volume': 'volume'
        })
        
        # Verificar colunas essenciais
        colunas_necessarias = ['open', 'high', 'low', 'close', 'volume']
        if not all(col in dados.columns for col in colunas_necessarias):
            missing = [col for col in colunas_necessarias if col not in dados.columns]
            raise ValueError(f"Colunas faltantes: {missing}")
        
        # Adicionar colunas extras
        dados['adjusted_close'] = dados['close'].astype(float)
        dados['dividend_amount'] = 0.0
        dados['split_coefficient'] = 1.0
        dados['ticker'] = ticker + '.SA'
        
        return dados.reset_index().rename(columns={'Date': 'date'})
    
    except Exception as e:
        print(f"Erro ao processar {ticker}: {str(e)}")
        return None

def criar_features(dados, features_esperadas):
    """Cria features de forma segura e compatível"""
    if dados is None or dados.empty:
        return None
        
    df = dados.copy()
    
    try:
        # 1. Corrigir formato do DataFrame
        df = corrigir_dataframe(df)
        
        # 2. Features básicas
        setores = {'PETR4.SA': 'Energia', 'VALE3.SA': 'Mineração', 'ITUB4.SA': 'Financeiro'}
        df['sector'] = df['ticker'].map(setores)
        le = LabelEncoder()
        df['sector_encoded'] = le.fit_transform(df['sector'].fillna('Outros')).astype(float)
        
        # 3. Cálculo de retornos
        for window in [1, 5, 21, 63]:
            df[f'retorno_{window}D'] = df.groupby('ticker')['close'].pct_change().rolling(window).mean().values
        
        # 4. Volatilidade
        df['volatilidade_21D'] = df.groupby('ticker')['retorno_1D'].rolling(21).std().values
        
        # 5. Volume (cálculo seguro)
        df['volume_avg_21D'] = df.groupby('ticker')['volume'].rolling(21).mean().values
        df['volume_spike'] = np.where(
            df['volume_avg_21D'] > 0,
            df['volume'] / df['volume_avg_21D'],
            0
        )
        
        # 6. Features que não podem ser calculadas com 3 meses
        for feature in ['RSI_14', 'BB_upper', 'BB_lower', 'SMA_200']:
            if feature in features_esperadas:
                df[feature] = 0.0
                
        # 7. SMA_50 (se necessário)
        if 'SMA_50' in features_esperadas:
            df['SMA_50'] = df.groupby('ticker')['close'].rolling(50).mean().values
        
        # 8. Features de dividendos
        df['dividend_yield'] = 0.0
        df['dividend_payment'] = 0.0
        
        # 9. Features de setor (simplificadas)
        df['sector_avg_retorno_21D'] = df.groupby('sector')['retorno_21D'].transform('mean')
        df['sector_avg_volume'] = df.groupby('sector')['volume'].transform('mean')
        
        # Garantir todas as features esperadas
        for feature in features_esperadas:
            if feature not in df.columns:
                df[feature] = 0.0
        
        return df[features_esperadas]
    
    except Exception as e:
        print(f"Erro ao criar features: {str(e)}")
        return None

def prever_acao(ticker):
    try:
        # 1. Carregar modelo e features
        modelo = lgb.Booster(model_file='utils/modelo_lgbm_todas.txt')
        features_esperadas = pd.read_csv('utils/features_utilizadas_todas.csv', header=None)[0].tolist()
        
        # Remover possível cabeçalho incorreto
        if features_esperadas[0] == '0':
            features_esperadas = features_esperadas[1:]
        
        print(f"\nFeatures esperadas ({len(features_esperadas)}): {features_esperadas}")
        
        # 2. Obter dados
        dados = processar_dados(ticker)
        if dados is None:
            return None
            
        print("\nColunas disponíveis:", dados.columns.tolist())
        
        # 3. Criar features
        dados_features = criar_features(dados, features_esperadas)
        if dados_features is None:
            return None
            
        print("\nFeatures criadas:", dados_features.columns.tolist())
        
        # 4. Fazer previsão
        ultimo_dia = dados_features.iloc[[-1]].values.astype(float)
        retorno = float(modelo.predict(ultimo_dia)[0])
        
        # 5. Resultado
        print(f"\nPrevisão para {ticker}:")
        print(f"Data: {dados['date'].iloc[-1].strftime('%d/%m/%Y')}")
        print(f"Preço: R${dados['close'].iloc[-1]:.2f}")
        print(f"Retorno previsto: {retorno:.2%}")
        print(f"Preço projetado: R${dados['close'].iloc[-1] * (1 + retorno):.2f}")
        
        return retorno
        
    except Exception as e:
        print(f"Erro ao prever {ticker}: {str(e)}")
        return None

if __name__ == "__main__":
    acoes = ['MGLU3']
    resultados = []
    
    for acao in acoes:
        print(f"\nProcessando {acao}...")
        retorno = prever_acao(acao)
        if retorno is not None:
            resultados.append({
                'ticker': acao,
                'data': datetime.now().strftime('%Y-%m-%d'),
                'retorno_previsto': retorno,
                'preco_atual': yf.Ticker(acao + '.SA').history(period='1d')['Close'].iloc[-1]
            })
    
    if resultados:
        pd.DataFrame(resultados).to_csv('previsoes.csv', index=False)
        print("\nPrevisões salvas em 'previsoes.csv'")
    else:
        print("\nNenhuma previsão concluída com sucesso")


Processando MGLU3...

Features esperadas (17): ['retorno_1D', 'retorno_5D', 'retorno_21D', 'retorno_63D', 'volatilidade_21D', 'volume_avg_21D', 'volume_spike', 'RSI_14', 'SMA_50', 'SMA_200', 'BB_upper', 'BB_lower', 'dividend_yield', 'dividend_payment', 'sector_encoded', 'sector_avg_retorno_21D', 'sector_avg_volume']


C:\Users\Colaborador\AppData\Local\Temp\ipykernel_26500\3992977878.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dados = yf.download(ticker + '.SA', period='1y', progress=False)



Colunas disponíveis: ['date', 'close', 'high', 'low', 'open', 'volume', 'adjusted_close', 'dividend_amount', 'split_coefficient', 'ticker']

Features criadas: ['retorno_1D', 'retorno_5D', 'retorno_21D', 'retorno_63D', 'volatilidade_21D', 'volume_avg_21D', 'volume_spike', 'RSI_14', 'SMA_50', 'SMA_200', 'BB_upper', 'BB_lower', 'dividend_yield', 'dividend_payment', 'sector_encoded', 'sector_avg_retorno_21D', 'sector_avg_volume']

Previsão para MGLU3:
Data: 28/07/2025
Preço: R$7.18
Retorno previsto: 20.91%
Preço projetado: R$8.68

Previsões salvas em 'previsoes.csv'


In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from datetime import datetime, timedelta
from sklearn.base import BaseEstimator, TransformerMixin
import os

class FeatureExtractor(BaseEstimator, TransformerMixin):
    """Classe para extração avançada de features"""
    def __init__(self):
        self.features_to_use = []
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, df):
        df = df.copy()
        
        # Garantir ordenação correta
        if 'data_pregao' in df.columns:
            df['data_pregao'] = pd.to_datetime(df['data_pregao'])
            df = df.sort_values(['cod_negociacao', 'data_pregao'])
        
        # Grupo para cálculos
        gb = df.groupby('cod_negociacao')['preco_fechamento']
        
        # 1. Médias móveis e volatilidade
        for window in [4, 8, 12, 26]:
            df[f'media_{window}w'] = gb.rolling(window=window).mean().reset_index(level=0, drop=True)
            df[f'vol_{window}w'] = gb.rolling(window=window).std().reset_index(level=0, drop=True)
        
        # 2. Retornos protegidos contra divisão por zero
        def safe_return(current, past, min_price=0.01):
            with np.errstate(divide='ignore', invalid='ignore'):
                return np.where(
                    (past > min_price) & (current > min_price),
                    (current - past) / past,
                    np.nan
                )
        
        for weeks, periods in [(1,0), (4,3), (12,11), (26,25)]:
            preco_passado = gb.shift(periods)
            df[f'retorno_{weeks}w'] = safe_return(df['preco_fechamento'], preco_passado)
        
        # 3. RSI manual (sem TA-Lib)
        def calculate_rsi(series, window=14):
            delta = series.diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)
            
            avg_gain = gain.rolling(window=window, min_periods=1).mean()
            avg_loss = loss.rolling(window=window, min_periods=1).mean()
            
            rs = avg_gain / avg_loss.replace(0, np.nan)
            return 100 - (100 / (1 + rs.replace(np.inf, 100)))
        
        df['rsi_14'] = df.groupby('cod_negociacao')['preco_fechamento'].transform(calculate_rsi)
        
        # 4. Features temporais
        for lag in [1, 2, 3, 4]:
            df[f'retorno_lag_{lag}w'] = df.groupby('cod_negociacao')['retorno_1w'].shift(lag)
        
        df['ema_12'] = df.groupby('cod_negociacao')['preco_fechamento'].transform(
            lambda x: x.ewm(span=12).mean()
        )
        
        # 5. Features de volume
        if 'volume' in df.columns:
            vol_gb = df.groupby('cod_negociacao')['volume']
            for window in [4, 12]:
                df[f'volume_medio_{window}w'] = vol_gb.rolling(window=window).mean().reset_index(level=0, drop=True)
        
        # Definir features a serem usadas
        self.features_to_use = [
            'media_4w', 'media_12w', 'vol_4w', 'vol_12w',
            # 'retorno_1w',
            'retorno_4w', 'retorno_12w',
            # 'retorno_lag_1w', 'retorno_lag_2w',
            'volume_medio_4w', 'ema_12', 'rsi_14'
        ]
        
        return df

class StockPredictor:
    def __init__(self, model, feature_extractor):
        self.model = model
        self.feature_extractor = feature_extractor
    
    def get_data_from_yfinance(self, ticker, start_date, end_date):
        """
        Obtém dados históricos do Yahoo Finance
        """
        print(f"Obtendo dados do Yahoo Finance para {ticker}...")
        try:
            stock = yf.Ticker(ticker + ".SA")  # Adiciona .SA para ações brasileiras
            df = stock.history(start=start_date, end=end_date)
            
            if df.empty:
                raise ValueError(f"Nenhum dado encontrado para {ticker}")
            
            # Renomear colunas para corresponder ao formato esperado
            df = df.reset_index()
            df = df.rename(columns={
                'Date': 'data_pregao',
                'Close': 'preco_fechamento',
                'Volume': 'volume'
            })
            
            # Adicionar código de negociação
            df['cod_negociacao'] = ticker
            
            return df[['cod_negociacao', 'data_pregao', 'preco_fechamento', 'volume']]
        
        except Exception as e:
            print(f"Erro ao obter dados do Yahoo Finance: {str(e)}")
            return None
    
    def get_data_from_brapi(self, ticker, start_date, end_date):
        """
        Obtém dados históricos da Brapi.dev
        """
        print(f"Obtendo dados da Brapi.dev para {ticker}...")
        try:
            url = f"https://brapi.dev/api/quote/{ticker}?interval=1d&start={start_date}&end={end_date}"
            response = requests.get(url)
            data = response.json()
            
            if 'results' not in data or not data['results']:
                raise ValueError(f"Nenhum dado encontrado para {ticker}")
            
            # Extrair dados históricos
            historical_data = data['results'][0]['historicalDataPrice']
            
            # Converter para DataFrame
            df = pd.DataFrame(historical_data)
            
            # Converter timestamp para data
            df['data_pregao'] = pd.to_datetime(df['date'], unit='s').dt.date
            
            # Renomear colunas
            df = df.rename(columns={
                'close': 'preco_fechamento',
                'volume': 'volume'
            })
            
            # Adicionar código de negociação
            df['cod_negociacao'] = ticker
            
            return df[['cod_negociacao', 'data_pregao', 'preco_fechamento', 'volume']]
        
        except Exception as e:
            print(f"Erro ao obter dados da Brapi.dev: {str(e)}")
            return None
    
    def prepare_data(self, df):
        """
        Prepara os dados para previsão usando o FeatureExtractor
        """
        # Garantir ordenação por data
        df = df.sort_values(['cod_negociacao', 'data_pregao'])
        
        # Extrair features
        df_features = self.feature_extractor.transform(df)
        
        # Selecionar apenas as features usadas pelo modelo
        X = df_features[self.feature_extractor.features_to_use]
        
        return X
    
    def predict(self, ticker, source='yfinance', days_back=260):
        """
        Faz previsões para um determinado ticker
        """
        # Definir datas
        end_date = datetime.now().strftime('%Y-%m-%d')
        start_date = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')
        
        # Obter dados
        if source.lower() == 'yfinance':
            df = self.get_data_from_yfinance(ticker, start_date, end_date)
        elif source.lower() == 'brapi':
            df = self.get_data_from_brapi(ticker, start_date, end_date)
        else:
            raise ValueError("Fonte de dados inválida. Use 'yfinance' ou 'brapi'")
        
        if df is None or df.empty:
            print(f"Não foi possível obter dados para {ticker}")
            return None
        
        # Preparar dados
        X = self.prepare_data(df)
        
        if X is None or X.empty:
            print("Não foi possível preparar os dados para previsão")
            return None
        
        # Fazer previsões
        predictions = self.model.predict(X)
        
        # Criar DataFrame com resultados
        results = df[['cod_negociacao', 'data_pregao', 'preco_fechamento']].copy()
        results['retorno_previsto_52semanas'] = predictions
        
        # Adicionar data de referência para o retorno futuro
        results['data_referencia_retorno'] = results['data_pregao'] + pd.to_timedelta(52*7, unit='days')
        
        return results
    
    def predict_multiple(self, tickers, source='yfinance', days_back=260):
        """
        Faz previsões para múltiplos tickers
        """
        all_results = []
        
        for ticker in tickers:
            print(f"\nProcessando {ticker}...")
            result = self.predict(ticker, source, days_back)
            
            if result is not None:
                all_results.append(result)
        
        if not all_results:
            return None
        
        return pd.concat(all_results)

def load_feature_extractor(file_path):
    try:
        # Verificar se arquivo existe
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Arquivo {file_path} não encontrado")
            
        # Carregar com tratamento de erro
        with open(file_path, 'rb') as file:
            return pickle.load(file)
            
    except AttributeError as e:
        print("\n⚠️ Erro: Classe FeatureExtractor não encontrada. Soluções possíveis:")
        print("1. Certifique-se que a classe FeatureExtractor está definida no seu script atual")
        print("2. Se você modificou a classe, recrie e salve o feature extractor novamente")
        print(f"Detalhes do erro: {str(e)}")
        return None
    except Exception as e:
        print(f"\n❌ Erro ao carregar feature extractor: {str(e)}")
        return None

def get_expected_annual_return(ticker, model_path='utils/modelo_xgb_treinado_2017_2018.json', 
                              feature_extractor_path='utils/feature_extractor_2017_2018.pkl'):
    """
    Retorna a rentabilidade anual esperada para uma ação específica
    
    Args:
        ticker (str): Código da ação (ex: 'PETR4')
        model_path (str): Caminho para o modelo XGBoost treinado
        feature_extractor_path (str): Caminho para o feature extractor
        
    Returns:
        float: Rentabilidade anual esperada em percentual (ex: 0.15 para 15%)
        str: Mensagem de status
    """
    
    # 1. Carregar o modelo e feature extractor
    try:
        modelo = XGBRegressor()
        modelo.load_model(model_path)
        
        with open(feature_extractor_path, 'rb') as f:
            feature_extractor = pickle.load(f)
    except Exception as e:
        return None, f"Erro ao carregar modelo: {str(e)}"
    
    # 2. Obter dados mais recentes
    try:
        stock = yf.Ticker(ticker + ".SA")
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365)  # 1 ano de dados
        
        df = stock.history(start=start_date, end=end_date)
        
        if df.empty:
            return None, "Nenhum dado encontrado para o ticker"
            
        # Preparar dados no formato esperado
        df = df.reset_index().rename(columns={
            'Date': 'data_pregao',
            'Close': 'preco_fechamento',
            'Volume': 'volume'
        })
        df['cod_negociacao'] = ticker
    except Exception as e:
        return None, f"Erro ao obter dados: {str(e)}"
    
    # 3. Preparar features
    try:
        df_features = feature_extractor.transform(df)
        X = df_features[feature_extractor.features_to_use].iloc[[-1]]  # Pegar apenas o último dia
    except Exception as e:
        return None, f"Erro ao preparar features: {str(e)}"
    
    # 4. Fazer previsão
    try:
        pred = modelo.predict(X)
        annual_return = pred[0]  # Retorno esperado em decimal (ex: 0.15 para 15%)
        return annual_return, "Previsão realizada com sucesso"
    except Exception as e:
        return None, f"Erro ao fazer previsão: {str(e)}"


# Exemplo de uso:
if __name__ == "__main__":
    import pickle
    from xgboost import XGBRegressor
    # Supondo que você já tenha o modelo e o feature extractor
    # (substitua por como você os obtém na sua implementação)
    
    # Carregar o modelo e feature extractor (substitua por sua implementação real)
    # modelo_xgb, X_data, y_data = main("datasets/semanal2021-2022.csv")
    # feature_extractor = FeatureExtractor()
    
    ticker = 'BBDC4'
    
    retorno, mensagem = get_expected_annual_return(ticker)
    
    if retorno is not None:
        print(f"\nRentabilidade anual esperada para {ticker}: {retorno:.2%}")
        print(f"Status: {mensagem}")
    else:
        print(f"\nErro: {mensagem}")


Rentabilidade anual esperada para BBDC4: -4.79%
Status: Previsão realizada com sucesso


In [2]:
import numpy
print(numpy.__version__)

import pandas
print(pandas.__version__)

import sklearn
print(sklearn.__version__)



2.3.0
2.3.0
1.7.1


In [12]:
print(resultados)

    cod_negociacao               data_pregao  preco_fechamento  \
0            PETR4 2024-11-11 00:00:00-03:00         31.811728   
1            PETR4 2024-11-12 00:00:00-03:00         32.408470   
2            PETR4 2024-11-13 00:00:00-03:00         32.364597   
3            PETR4 2024-11-14 00:00:00-03:00         32.706841   
4            PETR4 2024-11-18 00:00:00-03:00         33.522976   
..             ...                       ...               ...   
168          PETR4 2025-07-21 00:00:00-03:00         31.049999   
169          PETR4 2025-07-22 00:00:00-03:00         31.350000   
170          PETR4 2025-07-23 00:00:00-03:00         31.990000   
171          PETR4 2025-07-24 00:00:00-03:00         31.940001   
172          PETR4 2025-07-25 00:00:00-03:00         31.980000   

     retorno_previsto_52semanas   data_referencia_retorno  
0                      0.336298 2025-11-10 00:00:00-03:00  
1                      0.336298 2025-11-11 00:00:00-03:00  
2                      0.33